In [1]:
import uproot
import numpy as np
import torch
import awkward as ak
import h5py

from tensorflow.keras.models import load_model

from HigherTier import HigherTierFileHelper
import GraphBuilder
import Models
import MPNN

In [2]:
#######################################
# Create our models
#######################################

############################
# Config 
############################
primaryFileSuffix = '_ParticleToNeutrinoTruth.pt'

# For primary GNN
NUM_LAYERS_GNN = 4
EMB_DIM_GNN = 16
INPUT_DIM_GNN = 23 # node features
EDGE_DIM_GNN = 1
GNN_MODEL_PATH = "/Users/isobel/Desktop/DUNE/2024/Hierarchy/models/gnn_" + primaryFileSuffix

# For primary edge classification model
PRIMARY_EDGE_CLASSIFIER_MODEL_PATH = "/Users/isobel/Desktop/DUNE/2024/Hierarchy/models/edge_classifier_" + primaryFileSuffix
OTHER_EDGE_CLASSIFIER_MODEL_PATH = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/models/other_simple_model'

############################
# Setup the models for eval
############################

gnn_model = MPNN.MPNNModel(num_layers=NUM_LAYERS_GNN, emb_dim=EMB_DIM_GNN, input_dim=INPUT_DIM_GNN, edge_dim=EDGE_DIM_GNN)
primary_edge_classifier_model = Models.EdgeClassifier()

gnn_model.load_state_dict(torch.load(GNN_MODEL_PATH))
gnn_model.eval()

primary_edge_classifier_model.load_state_dict(torch.load(PRIMARY_EDGE_CLASSIFIER_MODEL_PATH))
primary_edge_classifier_model.eval()

other_edge_classifier_model = load_model(OTHER_EDGE_CLASSIFIER_MODEL_PATH)

In [3]:
#############################
# Let's read the file
#############################

#fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/nu_dune10kt_1x2x6_1413_998_20230826T184801Z_gen_g4_detsim_hitreco__20240229T163039Z_reco2_ccnutree'

fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/without2DPFPs/ccnutree_0'

inputFileName = fileName + '.root'

treeFile = uproot.open(inputFileName)
tree = treeFile['ccnuselection/ccnusel']
branches = tree.arrays()

In [4]:
#############################
# Get event-level stuff
#############################
run = np.array(branches['Run'])
subrun = np.array(branches['SubRun'])
event = np.array(branches['Event'])

recoNuVertexX = branches['RecoNuVtxX']
recoNuVertexY = branches['RecoNuVtxY']
recoNuVertexZ = branches['RecoNuVtxZ']


recoGeneration_main = branches['RecoPFPRecoGeneration']
    
#############################
# Get pfp-level stuff - these cannot be numpy arrays...
#############################
trackShowerScore_main = branches['RecoPFPTrackShowerScore']
nHits_main = branches['RecoPFPRecoNHits']
charge_main = branches['RecoPFPRecoCharge']
vertexX_main = branches['RecoPFPRecoVertexX']
vertexY_main = branches['RecoPFPRecoVertexY']
vertexZ_main = branches['RecoPFPRecoVertexZ']
trackEndX_main = branches['RecoTrackRecoEndX']
trackEndY_main = branches['RecoTrackRecoEndY']
trackEndZ_main = branches['RecoTrackRecoEndZ']
showerDirX_main = branches['RecoShowerRecoDirX']  # not the best direction estimate, placeholder
showerDirY_main = branches['RecoShowerRecoDirY']
showerDirZ_main = branches['RecoShowerRecoDirZ']
ivysaurusMuon_main = branches['RecoPFPIvysaurusMuon']
ivysaurusProton_main = branches['RecoPFPIvysaurusProton']
ivysaurusPion_main = branches['RecoPFPIvysaurusPion']
ivysaurusElectron_main = branches['RecoPFPIvysaurusElectron']
ivysaurusPhoton_main = branches['RecoPFPIvysaurusPhoton']
wiggliness_main = branches['RecoTrackDeflecAngleSD']
trackLength_main = branches['RecoTrackLength']
displacement_main = branches['RecoShowerPandrizzleDisplacement']
dca_main = branches['RecoShowerPandrizzleDCA']
recoGeneration_main = branches['RecoPFPRecoGeneration']
recoSelf_main = branches['RecoPFPSelf']
pathwayLength_main = branches['RecoShowerPandrizzlePathwayLengthMin']
pathwayScatteringAngle2D_main = branches['RecoShowerPandrizzleMaxShowerStartPathwayScatteringAngle2D']
nShowerStartHits_main = branches['RecoShowerPandrizzleMaxNPostShowerStartHits']
showerScatterAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartScatterAngle']
nuVertexEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyAsymmetry']
showerStartEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartShowerStartEnergyAsymmetry']
nuVertexEnergyWeightedMeanRadialDistance_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyWeightedMeanRadialDistance']
showerMoliereRadius_main = branches['RecoShowerPandrizzleMinPostShowerStartShowerStartMoliereRadius']
showerOpeningAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartOpeningAngle']
foundHitRatio_main = branches['RecoShowerPandrizzleMaxFoundHitRatio']
initialGapSize_main = branches['RecoShowerPandrizzleMaxInitialGapSize']
largestProjectedGapSize_main = branches['RecoShowerPandrizzleMinLargestProjectedGapSize']
nViewsWithAmbiguousHits_main = branches['RecoShowerPandrizzleNViewsWithAmbiguousHits']
ambiguousHitMaxUnaccountedEnergy_main = branches['RecoShowerPandrizzleAmbiguousHitMaxUnaccountedEnergy']

# True information.. (for cheating)
pfpTrueMomX_main = branches['RecoPFPTrueMomX']
pfpTrueMomY_main = branches['RecoPFPTrueMomY']
pfpTrueMomZ_main = branches['RecoPFPTrueMomZ']
pfpTruePDG_main = branches['RecoPFPTruePDG']

#############################
# Network truth - these cannot be numpy arrays...
#############################
trueVisibleGeneration_main = branches['RecoPFPTrueVisibleGeneration']
trueTrackID_main = branches['RecoPFPTrueTrackID']
trueVisibleParentTrackID_main = branches['RecoPFPTrueVisibleParentTrackID']

#############################
# How many entries are we working with?
#############################
nEntries = run.shape[0]
print('We are working with:', nEntries, 'entries')

#############################
# Network scores
#############################
trueTrackID_out = []
trueVisibleGeneration_out = []
trueVisibleParentTrackID_out = []

particleIndex_out = []
primaryScores_out = []

higherTierChildIndex_out = []
higherTierParentIndex_out = []
higherTierParentTrackID_out = []
higherTierScores_out = []

We are working with: 98400 entries


In [5]:
###############################################
# Find Primaries - Let's create our graphs!
###############################################

modeDict = {
    "ADD_NEUTRINO"                 : True,
    "CHEAT_DIRECTION"              : True,
    "CHEAT_PID"                    : True, 
    "MAKE_PARTICLE_PARTICLE_LINKS" : True,
    "EDGE_FRACTION"                : 0.8,
    "DO_NORMALISATION"             : True,
    "IS_PRIMARY_TRAINING"          : False,
    "IS_HIGHER_TIER_TRAINING"      : False,
    "MAX_NODE_CLASS"               : 4
}

for iEvent in range(10) : 
    
    trueTrackID_event = []
    trueVisibleGeneration_event = []
    trueVisibleParentTrackID_event = []

    particleIndex_event = []
    primaryScores_event = []

    higherTierChildIndex_event = []
    higherTierParentIndex_event = []
    higherTierScores_event = []
            
    variables, y, parentPFPIndices, childPFPIndices = HigherTierFileHelper.readEvent(inputFileName, iEvent)
        
    eventDict = {
        "recoNuVertexX"                            : recoNuVertexX[iEvent], \
        "recoNuVertexY"                            : recoNuVertexY[iEvent], \
        "recoNuVertexZ"                            : recoNuVertexZ[iEvent], \
        "nParticles"                               : np.array(pfpTruePDG_main[iEvent]).shape[0], \
        "trackShowerScore"                         : np.array(trackShowerScore_main[iEvent], dtype='f'), \
        "nHits"                                    : np.array(nHits_main[iEvent], dtype='f'), \
        "charge"                                   : np.array(charge_main[iEvent], dtype='f'), \
        "vertexX"                                  : np.array(vertexX_main[iEvent], dtype='f'), \
        "vertexY"                                  : np.array(vertexY_main[iEvent], dtype='f'), \
        "vertexZ"                                  : np.array(vertexZ_main[iEvent], dtype='f'), \
        "trackEndX"                                : np.array(trackEndX_main[iEvent], dtype='f'), \
        "trackEndY"                                : np.array(trackEndY_main[iEvent], dtype='f'), \
        "trackEndZ"                                : np.array(trackEndZ_main[iEvent], dtype='f'), \
        "showerDirX"                               : np.array(showerDirX_main[iEvent], dtype='f'), \
        "showerDirY"                               : np.array(showerDirY_main[iEvent], dtype='f'), \
        "showerDirZ"                               : np.array(showerDirZ_main[iEvent], dtype='f'), \
        "ivysaurusMuon"                            : np.array(ivysaurusMuon_main[iEvent], dtype='f'), \
        "ivysaurusProton"                          : np.array(ivysaurusProton_main[iEvent], dtype='f'), \
        "ivysaurusPion"                            : np.array(ivysaurusPion_main[iEvent], dtype='f'), \
        "ivysaurusElectron"                        : np.array(ivysaurusElectron_main[iEvent], dtype='f'), \
        "ivysaurusPhoton"                          : np.array(ivysaurusPhoton_main[iEvent], dtype='f'), \
        "trackLength"                              : np.array(trackLength_main[iEvent], dtype='f'), \
        "displacement"                             : np.array(displacement_main[iEvent], dtype='f'), \
        "dca"                                      : np.array(dca_main[iEvent], dtype='f'), \
        "isNeutrinoPDG"                            : np.zeros(np.array(pfpTruePDG_main[iEvent]).shape), \
        "nuVertexEnergyAsymmetry"                  : np.array(nuVertexEnergyAsymmetry_main[iEvent], dtype='f'), \
        "nuVertexEnergyWeightedMeanRadialDistance" : np.array(nuVertexEnergyWeightedMeanRadialDistance_main[iEvent], dtype='f'), \
        "trueTrackID"                              : np.array(trueTrackID_main[iEvent]), \
        "trueVisibleParentTrackID"                 : np.array(trueVisibleParentTrackID_main[iEvent]), \
        "trueMomX"                                 : np.array(pfpTrueMomX_main[iEvent]), \
        "trueMomY"                                 : np.array(pfpTrueMomY_main[iEvent]), \
        "trueMomZ"                                 : np.array(pfpTrueMomZ_main[iEvent]), \
        "truePDG"                                  : np.array(pfpTruePDG_main[iEvent]), \
        "trueVisibleGeneration"                    : np.array(trueVisibleGeneration_main[iEvent])
    }

    data_pos, data_neg, data_FC, pfp_index = GraphBuilder.GraphBuilder(eventDict, modeDict)
    
    ##################################################
    # Calculate Scores of neutrino -> particle edges
    ##################################################
    if (data_FC.num_nodes > 1) :
        pred = gnn_model(data_FC)

        # Assuming that the neutrino is the last node added (which it is in this config)
        nuEdgeMask = (data_FC.edge_index[0] == (data_FC.num_nodes - 1))
        target_index = np.reshape(data_FC.edge_index[0][nuEdgeMask].detach().numpy(), -1)
        source_index = np.reshape(data_FC.edge_index[1][nuEdgeMask].detach().numpy(), -1)
        edge_index = torch.tensor([target_index, source_index], dtype=torch.long)

        edge_pred = primary_edge_classifier_model(pred, edge_index)

    for iParticle in range(eventDict["nParticles"]) :
    
        # Move on if it is the neutrino
        if (eventDict['isNeutrinoPDG'][iParticle] == 1) :
            continue
            
        # Node is not in graph (inf ivy or 2D?)
        if (len(np.where(pfp_index == iParticle)[0]) == 0) :
            continue 
    
        primary_GNN_score = edge_pred[np.where(pfp_index == iParticle)[0][0]].item()
                
        trueTrackID_event.append(trueTrackID_main[iEvent][iParticle])
        trueVisibleGeneration_event.append(trueVisibleGeneration_main[iEvent][iParticle])
        trueVisibleParentTrackID_event.append(trueVisibleParentTrackID_main[iEvent][iParticle])
        particleIndex_event.append(iParticle)
        primaryScores_event.append(primary_GNN_score)

    ############################
    # Higher Tiers
    ############################    
    for iChild in range(eventDict["nParticles"]) :
            
        # We don't care about the neutrino here
        if (eventDict["isNeutrinoPDG"][iChild] == 1) :
            continue
            
        # Only look at 3D particles
        if (vertexX_main[iEvent][iChild] < -900) :
            continue
            
        for iParent in range(eventDict["nParticles"]) :
                
            if (eventDict["isNeutrinoPDG"][iParent] == 1) :
                continue
                
            # Only look at 3D particles
            if (vertexX_main[iEvent][iParent] < -900) :
                continue
                
            # Need to add more to the CCNuSelection
            linkIndex = HigherTierFileHelper.getLinkIndex(parentPFPIndices, childPFPIndices, iParent, iChild) 

            # If the particle-particle link hasn't been saved in the analyser
            if (linkIndex < 0) :
                print('ISOBEL SOMETHING HAS GONE WRONG - SAD')
                continue
                    
            linkVariables = variables[linkIndex].reshape(-1,12)
                                
            y_pred = other_edge_classifier_model.predict(linkVariables, verbose=2)
                        
            higherTierChildIndex_event.append(iChild)
            higherTierParentIndex_event.append(iParent)
            higherTierScores_event.append(y_pred[0][0])
            
            
#     print('//////////////////////////////////')
#     print('//////////// NEW EVENT ///////////')
#     print('//////////////////////////////////')
#     print('trueVisibleGeneration_event:', trueVisibleGeneration_event)
#     print('trueVisibleParentTrackID_event:', trueVisibleParentTrackID_event)
#     print('particleIndex_event:', particleIndex_event)
#     print('primaryScores_event:', primaryScores_event)
#     print('higherTierChildIndex_event:', higherTierChildIndex_event)
#     print('higherTierParentIndex_event:', higherTierParentIndex_event)
#     print('higherTierScores_event:', higherTierScores_event)
            

    trueTrackID_out.append(trueTrackID_event)
    trueVisibleGeneration_out.append(trueVisibleGeneration_event)
    trueVisibleParentTrackID_out.append(trueVisibleParentTrackID_event)
    particleIndex_out.append(particleIndex_event)
    primaryScores_out.append(primaryScores_event)
    higherTierChildIndex_out.append(higherTierChildIndex_event)
    higherTierParentIndex_out.append(higherTierParentIndex_event)
    higherTierScores_out.append(higherTierScores_event)
    
# print('//////////////////////////////////')
# print('//////////////////////////////////')
# print('trueVisibleGeneration_out:', trueVisibleGeneration_out)
# print('trueVisibleParentTrackID_out:', trueVisibleParentTrackID_out)
# print('particleIndex_out:', particleIndex_out)
# print('primaryScores_out:', primaryScores_out)
# print('higherTierChildIndex_out:', higherTierChildIndex_out)
# print('higherTierParentIndex_out:', higherTierParentIndex_out)
# print('higherTierScores_out:', higherTierScores_out)

ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMET

/var/folders/1g/qpfx8_ps0dn6405cqjp61k5r0000gn/T/ipykernel_44043/2121482672.py:81: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  edge_index = torch.tensor([target_index, source_index], dtype=torch.long)


1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
ISOBEL SOMETHING HAS GONE WRONG - SAD
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOBEL SOMETHING HAS GONE WRONG - SAD
ISOB

In [6]:
########################################################
# Python is annoying so we will have to pad our vectors
########################################################

def get_max_length(input_array) :
    
    lengths = [len(entry) for entry in input_array]
    lengths = np.array(lengths)
    
    return np.max(lengths)


def create_mask(input_array, max_len):
    
    file_mask = [True] * len(input_array)
    to_fill = [False] * (max_len - len(file_mask))
    file_mask = file_mask + to_fill
    
    return file_mask

def pad_array(input_array, max_len):
    
    pad = [0] * (max_len - len(input_array))
    input_array = input_array + pad
    
    return input_array

In [7]:
####################
# Create file mask
####################
particle_max_length = get_max_length(primaryScores_out)
particle_mask_out = [create_mask(entry, particle_max_length) for entry in primaryScores_out]
particle_mask_out = np.array(particle_mask_out)

link_max_length = get_max_length(higherTierScores_out)
link_mask_out = [create_mask(entry, link_max_length) for entry in higherTierScores_out]
link_mask_out = np.array(link_mask_out)

# print('--------- BEFORE ----------')
# print('trueTrackID_out:', trueTrackID_out)
# print('trueVisibleGeneration_out:', trueVisibleGeneration_out)
# print('trueVisibleParentTrackID_out:', trueVisibleParentTrackID_out)
# print('particleIndex_out:', particleIndex_out)
# print('primaryScores_out:', primaryScores_out)
# print('higherTierChildIndex_out:', higherTierChildIndex_out)
# print('higherTierParentIndex_out:', higherTierParentIndex_out)
# print('higherTierScores_out:', higherTierScores_out)
# print('---------------------------')

####################
# Pad vectors
####################
trueTrackID_out = [pad_array(entry, particle_max_length) for entry in trueTrackID_out]
trueTrackID_out = np.array(trueTrackID_out)

trueVisibleGeneration_out = [pad_array(entry, particle_max_length) for entry in trueVisibleGeneration_out]
trueVisibleGeneration_out = np.array(trueVisibleGeneration_out)

trueVisibleParentTrackID_out = [pad_array(entry, particle_max_length) for entry in trueVisibleParentTrackID_out]
trueVisibleParentTrackID_out = np.array(trueVisibleParentTrackID_out)

particleIndex_out = [pad_array(entry, particle_max_length) for entry in particleIndex_out]
particleIndex_out = np.array(particleIndex_out)

primaryScores_out = [pad_array(entry, particle_max_length) for entry in primaryScores_out]
primaryScores_out = np.array(primaryScores_out)

higherTierChildIndex_out = [pad_array(entry, link_max_length) for entry in higherTierChildIndex_out]
higherTierChildIndex_out = np.array(higherTierChildIndex_out)

higherTierParentIndex_out = [pad_array(entry, link_max_length) for entry in higherTierParentIndex_out]
higherTierParentIndex_out = np.array(higherTierParentIndex_out)

higherTierScores_out = [pad_array(entry, link_max_length) for entry in higherTierScores_out]
higherTierScores_out = np.array(higherTierScores_out)


# print('--------- AFTER ----------')
# print('trueTrackID_out:', trueTrackID_out)
# print('trueVisibleGeneration_out:', trueVisibleGeneration_out)
# print('trueVisibleParentTrackID_out:', trueVisibleParentTrackID_out)
# print('particleIndex_out:', particleIndex_out)
# print('primaryScores_out:', primaryScores_out)
# print('higherTierChildIndex_out:', higherTierChildIndex_out)
# print('higherTierParentIndex_out:', higherTierParentIndex_out)
# print('higherTierScores_out:', higherTierScores_out)
# print('---------------------------')

In [8]:
#############################
# Write out!
#############################

file = 'networkScores'
outputFile = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/without2DPFPs/' + file + '.npz'
    
np.savez(outputFile, \
         particle_mask=particle_mask_out,\
         trueTrackID=trueTrackID_out, \
         trueVisibleGeneration=trueVisibleGeneration_out, \
         trueVisibleParentTrackID=trueVisibleParentTrackID_out, \
         particleIndex=particleIndex_out, \
         primaryScores=primaryScores_out, \
         link_mask=link_mask_out,\
         higherTierChildIndex=higherTierChildIndex_out, \
         higherTierParentIndex=higherTierParentIndex_out, \
         higherTierScores=higherTierScores_out)